In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
import math
import copy

from datetime import datetime

import math
import mplcyberpunk

In [ ]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ipywidgets as widgets

In [ ]:
# maps
import folium
import gmaps.geojson_geometries

In [ ]:
# styyle/display stuff
from IPython.display import Markdown as md
import mplcyberpunk
import matplotlib.ticker as ticker
import seaborn as sns
from matplotlib import animation as F
from IPython.display import HTML, Image

In [ ]:
# models
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Bidirectional
from keras.optimizers import Adam

In [ ]:
import plotly as py
import plotly.express as px

In [ ]:
# score
from sklearn.metrics import mean_squared_error, r2_score, make_scorer


In [ ]:
# scaling & normalization
from sklearn.preprocessing import Normalizer, PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

If you wish to run this locally you will need the source file which can be found at <a href="https://github.com/nlucian/covid19/blob/master/BigDataProject_final.ipynb" > github repo </a>

### prerequisites  for running locally

conda install -c conda-forge voila <br/>
conda install folium -c conda-forge <br/>
conda install -c conda-forge gmaps <br/>
<br/>
pip install mplcyberpunk
conda update -all


update all the project dependencies to the latest version

### Folium
##### Documentation
https://python-visualization.github.io/folium/

### Voila
##### Documentation
https://voila.readthedocs.io/en/latest/?badge=latest

### pandas configuration

In [ ]:
# pandas global configuration
def set_pandas_display_options() -> None:
    display = pd.options.display

    display.max_columns = 100
    display.max_rows = 500
    display.max_colwidth = 200
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()

In [ ]:
#utility functions

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def applyStandardScaler(values):
    scaler = Normalizer()
    scaled_result = scaler.fit_transform(values)
    return scaled_result


In [ ]:
# date is currently seen as string
def StringisNaN(string):
    return string != string

# apply log to any number
def apply_log(x):
    if x != 0:
        return np.log(x)

<h4>We use the following data sources</h4>


<ul>
    <li><a href="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv">covid confirmed cases</a></li>
    <li><a href="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv">covid deaths timeseries</a></li>
    <li><a href="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv">covid recovered cases</a></li>
    <li><a href="https://raw.githubusercontent.com/nlucian/covid19/master/datasets/lockdown_country_dates.csv"> covid lockdown dates</a></li>
</ul>

In [ ]:
# data loading and processing

In [ ]:
numbers_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
numbers_dead = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
numbers_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

### data cleanup

In [ ]:
# uncomment if you wish to view the rows with misssing values
# missing_data_confirmedCases = numbers_confirmed.isnull()
# missing_data_confirmedCases.head(10)

 #### Only the Province column contains nan values - 182 such values

In [ ]:
lockdown_dates = pd.read_csv('https://raw.githubusercontent.com/nlucian/covid19/master/datasets/lockdown_country_dates.csv')
lockdown_Romania=lockdown_dates.loc[lockdown_dates['Country/Region'] == 'Romania']

In [ ]:
# this part is unpivotting the table from the wide format to the long format
confirmed_long = numbers_confirmed.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"], var_name="Date", value_name="Confirmed").fillna('')
dead_long = numbers_dead.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"], var_name="Date", value_name="Deaths").fillna('')
recovered_long = numbers_recovered.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"], var_name="Date", value_name="Recovered").fillna('')

In [ ]:
# turn the 3 different tables into a single table with all the information, confirmed, dead and recovered

daily_info = dead_long.merge(confirmed_long).merge(recovered_long).drop(['Lat', 'Long'], axis=1)
daily_info.tail(5)

daily_sum=daily_info.groupby(['Date'],as_index=True).sum()
daily_sum.tail(5)

#calculate daily values of deaths, confirmed and recovered 
daily_values=daily_sum.diff(periods=1)

### world daily values combined

In [ ]:
daily_values.tail(5)

In [ ]:
daily_info['Date'] = pd.to_datetime(daily_info['Date'])

In [ ]:
# project constant values
# the dataset is updated daily - we need the values of confirmed cases for the last date

CONFIRMED_last_date = numbers_confirmed.columns.to_list()[-1]
DEATHS_last_date    = numbers_dead.columns.to_list()[-1]
RECOVERED_last_date = numbers_recovered.columns.to_list()[-1]

In [ ]:
# data preparation for maps

In [ ]:
# we only need the country and the total cases for now
df1 = numbers_confirmed.loc[:,'Province/State' : 'Long']
df2 = numbers_confirmed.iloc[:,-1]

frames = [df1, df2]

confirmed_cases_temp = pd.concat(frames, axis = 1)

In [ ]:
confirmed_cases = confirmed_cases_temp.groupby(['Country/Region'], 
                                               as_index=False)[CONFIRMED_last_date].sum()

In [ ]:
array_of_provinces = {'Denmark': 'Greenland'}

# we want to define some provinces as individual countries
# since we already do a group by we also have to subtract the number of cases from the group by reuslts
def keep_provinces_as_countries(provinces, df_before_grp_by, df_after_grp_by, last_column_date_string):
    for country in provinces:
        print("_Merging into '{}' the values from '{}'.".format(country, provinces.get(country)))
        province_cases = df_before_grp_by[df_before_grp_by['Province/State'] == provinces.get(country)][last_column_date_string]
        
        province_cases_value = province_cases.to_list()[0]
        print("_Province_cases {}".format(province_cases_value))
        
        to_subtract_from = df_after_grp_by.loc[df_after_grp_by['Country/Region'] == country, [last_column_date_string]][last_column_date_string].to_list()[0]
        print("_value after group by {}".format(to_subtract_from))
        
        final_value = to_subtract_from - province_cases_value
        print("_Province cases final value {}".format(final_value))
        
        df_after_grp_by.loc[df_after_grp_by['Country/Region'] == country ,[last_column_date_string]]  = final_value
        
        print("_Appending country {} with cases {}".format(provinces.get(country), province_cases_value))
        return df_after_grp_by.append({'Country/Region' : provinces.get(country), str(CONFIRMED_last_date) : province_cases_value}, ignore_index=True)
    
confirmed_cases = keep_provinces_as_countries(array_of_provinces, confirmed_cases_temp, confirmed_cases, CONFIRMED_last_date)
    

In [ ]:
confirmed_cases = confirmed_cases.rename(columns = {'Country/Region': 'country', CONFIRMED_last_date: 'covid_cases'})

In [ ]:
confirmed_cases = confirmed_cases.sort_values(by = ['covid_cases'], ascending=False)

In [ ]:
# Load GeoJSON of countries
countries_geojson = gmaps.geojson_geometries.load_geometry('countries') 

In [ ]:
dict_confirmed_cases = pd.Series(confirmed_cases.covid_cases.values, index=confirmed_cases.country).to_dict()

### update keys to match the geojson namings

The left-hand side is the gmaps definition of countries which must be mapped to the github source of covid cases per country.

There are 3 cases:

There is either no  mapping at all since the github repo does not track the number of cases for the given country;
There is a name mismatching  which must be corrected;
The region/country in gmaps is part of a country so we cumulate the cases under the mother country;

eg.  if the name is wrong -> replacement of the github country name with the gmaps naming;
eg.  if some islands like Cayman Islands are owned by UK we display it under UK;
eg. no country present in github from the gmap list (since we iterate through all the countries, we just ignore it)

<ul>
    <li>Aland -> not found</li>
    <li>America Samoa ->  not found</li>
    <li>Antarctica -> not found</li>
    <li>French Southern and Antarctic Lands -> not found</li>
    <li>Northern Cyprus -> Cyprus</li>
    <li>Cayman Islands -> UK</li>
    <li>Curacao -> Netherlands</li>
    <li>The Bahamas -> Bahamas</li>
    <li>Repubulc of Congo -> Congo (Brazzavilli)</li>
    <li>Democratic Repubulc of the Congo -> Congo (Kinshasa)</li>
    <li>Comoros -> not found</li>
    <li>Cape Verde -> Cabo Verde</li>
    <li>Curaçao -> Netherlands</li>
    <li>Cayman Islands -> UK</li>
    <li>Falkland Islands -> UK</li>
    <li>Faroe Islands -> Denmark</li>
    <li>Federated States of Micronesia -> not found</li>
    <li>Guinea-Bissau -> Guinea Bissau</li>
    <li>Hong Kong S.A.R -> China</li>
    <li>Isli of Man -> UK</li>
    <li>Baykonur Cosmodrome -> not found</li>
    <li>Siachen Glacier -> not found</li>
    <li>South Korea -> Korea, South</li>
    <li>Lesotho -> not found</li>
    <li>North Macedonia -> Macedonia</li>
    <li>Myanmar -> Burma</li>
    <li>Northern Mariana Islands -> not found</li>
    <li>New Calidonia -> France</li>
    <li>Niue -> not found</li>
    <li>Palau -> not found</li>
    <li>North Korea -> Kim Jong is hiding these days </li>
    <li>Palistine -> not found</li>
    <li>French Polynesia -> France</li>
    <li>Solomon Islands -> not found</li>
    <li>Saint helina -> not found</li>
    <li>South Georgia and South Sandwich Islands -> not found</li>
    <li>Somaulland -> not found</li>
    <li>Saint Pierre and Miquelon,France -> France</li>
    <li>Repubulc of Serbia -> Serbia</li>
    <li>Swaziland -> not found</li>
    <li>Tajikistan -> not found</li>
    <li>East Timor -> Timor-Leste</li>
    <li>Turkmenistan -> not found</li>
    <li>Taiwan -> Taiwan*</li>
    <li>United States Virgin Islands-> UK</li>
    <li>Samoa -> not found</li>
    <li>Yemen -> not found</li>
    
<ul/>

In [ ]:
def replace_country_key(old_key, new_key, dictionary):
    try:
        dictionary[new_key] = dictionary[old_key]
        del dict_confirmed_cases[old_key]
    except Exception as e:
        print("Error while mapping country {} or country already mapped.".format(str(e)))

In [ ]:
# name mapping error
replace_country_key('US', 'United States of America', dict_confirmed_cases)
replace_country_key('Bahamas', 'The Bahamas', dict_confirmed_cases)
replace_country_key('Czechia', 'Czech Republic', dict_confirmed_cases)

replace_country_key('Congo (Brazzaville)', 'Republic of Congo', dict_confirmed_cases)
replace_country_key('Congo (Kinshasa)', 'Democratic Republic of the Congo', dict_confirmed_cases)

replace_country_key('Cabo Verde', 'Cape Verde', dict_confirmed_cases)
replace_country_key('Guinea-Bissau', 'Guinea Bissau', dict_confirmed_cases)
replace_country_key('Korea, South', 'South Korea', dict_confirmed_cases)
replace_country_key('North Macedonia', 'Macedonia', dict_confirmed_cases)
replace_country_key('Burma','Myanmar', dict_confirmed_cases)
replace_country_key('Serbia', 'Republic of Serbia', dict_confirmed_cases)
replace_country_key('Timor-Leste', 'East Timor', dict_confirmed_cases)
replace_country_key('Taiwan*', 'Taiwan', dict_confirmed_cases)
replace_country_key('Tanzania', 'United Republic of Tanzania', dict_confirmed_cases)

In [ ]:
# custom data for some of the next map trials

In [ ]:
dict_confirmed_cases
dict_confirmed_cases_df = pd.DataFrame(list(dict_confirmed_cases.items()), 
                                       columns = ['countries','covid_cases']) 

dict_confirmed_cases_df['countries'] = dict_confirmed_cases_df['countries'].astype(str)

# MAPS

### Worldmap with total COVID-19 cases

#### select the color you wish to generate the map with

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
dict_confirmed_cases_df['covid_cases'] =  dict_confirmed_cases_df['covid_cases'].astype(int)
dict_confirmed_cases_df['covid_cases'] =  dict_confirmed_cases_df['covid_cases'].apply(apply_log)

In [ ]:
def generate_covid_map(fill_color):

    legend_name = 'COVID-19 cases worldwide'
    m3 = folium.Map(location=[51.505, -0.09], tiles='cartodbpositron', max_bounds=True,
                   zoom_start=1.5, min_zoom = 2)

    folium.Choropleth(
        
        geo_data=countries_geojson,
        data=dict_confirmed_cases_df,
        fill_color=fill_color,
        legend_named='COVID-19 cases worldwide (log scale)',
        columns=['countries', 'covid_cases'],
        key_on='feature.properties.name',
        fill_opacity=0.9,
        line_opacity=0.1,
        nan_fill_color='ffffff'
    ).add_to(m3)
    
    display(m3)

In [ ]:
map_output_widget = widgets.Output()

In [ ]:
map_colors =  {'Green':'BuGn', 'Red':'RdPu'}

select_variable = widgets.Dropdown(
    options=map_colors,
    value=map_colors.get('Blue'),
    description='Colors'
)

def get_and_plot(b):
    with map_output_widget:
        clear_output()
        print(select_variable.value)
        generate_covid_map(select_variable.value)
        
select_variable.observe(get_and_plot, names='value')

display(select_variable)
display(map_output_widget)

### Worldmap with COVID-19 mortality 


In [ ]:
modified_deaths = numbers_dead[numbers_dead[DEATHS_last_date] > 0]

#### Density Plot

In [ ]:
plt.style.use("cyberpunk")
plt.subplots(figsize=(24, 10))
plt.suptitle('Density', fontsize = 25)
plt.subplot(1,2,1)
(modified_deaths[DEATHS_last_date]).plot.kde()
plt.subplot(1,2,2)
(np.log10(modified_deaths[DEATHS_last_date])).plot.kde()
plt.show()

### If we use the original values, then we won't be able to distinguish the colors of the map (that's why we are going apply $log_{10}$ since the values are more spread)

In [ ]:
modified_deaths['Transformed_value'] = np.log10(modified_deaths[DEATHS_last_date])

In [ ]:
df_new = pd.DataFrame(columns = ['Country', 'Date', 'Confirmed', 'Recovered', 'Dead', 'Transformed_value'])    

df_new['Country'] = modified_deaths['Country/Region']
df_new['Date'] = CONFIRMED_last_date
df_new['Confirmed'] = numbers_confirmed[CONFIRMED_last_date]
df_new['Dead'] = numbers_dead[DEATHS_last_date]
df_new['Recovered'] = numbers_recovered[RECOVERED_last_date]
df_new['Transformed_value'] = modified_deaths['Transformed_value']
df_new = df_new[::-1]

In [ ]:
fig = px.choropleth(
    df_new,
    locations = 'Country',
    locationmode = 'country names',
    color = 'Transformed_value',
    hover_name = 'Country',
    hover_data = ['Confirmed','Recovered','Dead'], 
    animation_frame= 'Date',
    color_continuous_scale=px.colors.diverging.RdYlGn[::-1]
)

fig.update_layout(
    title_text =   " COVID-19 Deaths " + DEATHS_last_date,
    title_x = 0.5,
    geo= dict(
        showframe= False,
        showcoastlines= False,
        projection_type = 'equirectangular'
    )
)

display(fig)

### healed vs deaths percentage

In [ ]:
# turn the 3 different tables into a single table with all the information, confirmed, dead and recovered
daily_info = dead_long.merge(confirmed_long).merge(recovered_long).drop(['Lat', 'Long'], axis=1)
daily_sum  = daily_info.groupby(['Date'],as_index=True).sum()

#calculate daily values of deaths, confirmed and recovered 
daily_values = daily_sum.diff(periods=1)

In [ ]:
daily_sum['death_percentage'] = daily_sum['Deaths']/daily_sum['Confirmed'] *100
daily_sum['heal_percentage']  = daily_sum['Recovered']/daily_sum['Confirmed'] *100

In [ ]:
daily_sum.reset_index(level=0, inplace=True)

In [ ]:
def plotHealDeathPercentage():   
        plt.style.use("cyberpunk")
        fig = plt.figure(figsize=(20,10))
        
        plt.locator_params(axis='x', nbins=15)
        
        plt.plot(daily_sum['Date'], daily_sum['death_percentage'], label = 'Death Percentage')
        plt.plot(daily_sum['Date'], daily_sum['heal_percentage'], label = 'Heal Percentage')
        
        plt.legend(loc=2, prop={'size': 18})
        plt.xticks(rotation=90, ha='right', fontsize="x-large")
        plt.yticks(fontsize="x-large")
        
        ax = plt.gca()
        temp = ax.xaxis.get_ticklabels()
        temp = list(set(temp) - set(temp[::3]))
        for label in temp:
            label.set_visible(False)
        
        
        mplcyberpunk.make_lines_glow(ax)


        plt.show()

plotHealDeathPercentage()

### Countries Investigated 

In [ ]:
# data modeling for the investigated countries plot

def dataFrameForCountry(country):
    country_data=daily_info.loc[daily_info['Country/Region'] == country]
    country_data=country_data.drop(["Province/State","Country/Region"], axis=1)
    country_data.set_index('Date', inplace = True) 

    country_data_daily = country_data.diff(periods=1)
    country_data_daily = country_data_daily[(country_data_daily > 0).any(axis=1)]

    country_data_daily['DateOf'] = country_data_daily.index
    return country_data_daily
    

In [ ]:
countries_Investigated = ['Romania', 'Italy', 'Sweden', 'Belarus','Iran','Belgium', 'Poland','Germany', 'Brazil','Mexico','Russia','Turkey','Indonesia','Egypt','Portugal','Ukraine','Pakistan','Hungary']
select_countries = widgets.Dropdown(
    options=countries_Investigated,
    value=countries_Investigated[0],
    description='Countries'
)

ci_output = widgets.Output()
display(widgets.VBox([ci_output, select_countries]))

In [ ]:
@ci_output.capture()
def RestrictionsEffectForCountry(country):
    plt.style.use("cyberpunk")
    
    dataFrame = dataFrameForCountry(country)
    
    dataFrame.plot(y=['Confirmed','Deaths','Recovered'],kind='bar',figsize=(20,10),color=['orange','darkred','lightgreen'])


    lockdown_date_type=lockdown_dates.loc[lockdown_dates['Country/Region'] == country]
  
    if(not(StringisNaN(lockdown_date_type.iloc[0]['Date']))):
        date_format = "%Y-%m-%d"
        
        first_lockdown_measure=str(datetime.strptime(lockdown_date_type.iloc[0]['Date'],date_format).date())
        dataFrame['DateOf'] = pd.to_datetime(dataFrame['DateOf'])
        plotdataFrame=dataFrame.reset_index()
        plotdataFrame["DateOf"] = plotdataFrame["DateOf"].apply(lambda x: datetime.combine(x.date(), datetime.min.time()))
        plotdataFrame["DateOf"] = plotdataFrame["DateOf"].dt.strftime("%Y-%m-%d")
        
        delta=plotdataFrame.loc[plotdataFrame["DateOf"] == first_lockdown_measure].index[0]
        
        plt.axvspan(0, delta, facecolor='#FFFF33', alpha=0.5,zorder=0.1)
        plt.axvspan(delta, len(dataFrame.index), facecolor='green', alpha=0.5,zorder=0.1)
        plt.suptitle(country+' (quarantine started on '+ first_lockdown_measure,fontsize=15)
    else:
        plt.suptitle(country+' (no quarantine)', fontsize=15)
    plt.legend(loc=1, prop={'size': 15})
    ax = plt.gca()
    temp = ax.xaxis.get_ticklabels()
    temp = list(set(temp) - set(temp[::3]))
    for label in temp:
        label.set_visible(False)
    x_axis = ax.axes.get_xaxis()
    x_axis.set_label_text('')
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    mplcyberpunk.make_lines_glow(ax)    
    plt.show()
    

In [ ]:
def plot_investigated_countries(a):
    with ci_output:
        clear_output()
        print(select_countries.value)
        RestrictionsEffectForCountry(select_countries.value)

select_countries.observe(plot_investigated_countries, names='value')

### General Information

In [ ]:
print("last update on the dataset: {}".format(CONFIRMED_last_date)) # CONFIRMED_last_date = numbers_confirmed.columns.to_list()[-1]

In [ ]:
total_confirmed_case = numbers_confirmed[CONFIRMED_last_date].sum()
print(f"{total_confirmed_case:,} confirmed cases")

total_recovered_cases = numbers_recovered[RECOVERED_last_date].sum()
print(f"{total_recovered_cases:,} recovered")

total_deaths = numbers_dead[RECOVERED_last_date].sum()
print(f"{total_deaths:,} deaths")

In [ ]:
top_ten = numbers_confirmed.sort_values(by = CONFIRMED_last_date  , ascending = False).head(10)

### Top 10 countries - confirmed cases

In [ ]:
top_ten['Country/Region']

### Top 10 countries with COVID-19 confirmed cases - last date

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))

#plt.figure(figsize=(12,8))
ax = sns.barplot(x = CONFIRMED_last_date, y = 'Country/Region', 
              data = top_ten)
ax.set(xlabel='Confirmed Cases', ylabel='Country')

for i, (value, name) in enumerate(zip(top_ten[CONFIRMED_last_date], top_ten['Country/Region'])):
    ax.text(value, i-0.20,     value,           ha='right')
    
ax.text(1, 0.4, CONFIRMED_last_date, transform=ax.transAxes, size=46, ha='right')

plt.title("Top ten countries with confirmed covid cases")

plt.show()

In [ ]:
nrc_grouped_by_country = numbers_confirmed.groupby(['Country/Region'], as_index=False).sum()
nrd_grouped_by_country = numbers_dead.groupby(['Country/Region'], as_index=False).sum()
nr_grouped_By_country = numbers_recovered.groupby(['Country/Region'], as_index=False).sum()

### Animated barcharts

In [ ]:
global category

category = 'Confirmed'

def draw_barchart(date):
    
    
    if category == 'Confirmed':
        df_top10 = (nrc_grouped_by_country.sort_values(by=date,ascending=False).head(10))
    elif category == 'Dead':
        df_top10 = (nrd_grouped_by_country.sort_values(by=date,ascending=False).head(10))
    elif category == 'Recovered':
        df_top10 = (nr_grouped_By_country.sort_values(by=date,ascending=False).head(10))
    else:
        return

    df_new = pd.DataFrame(columns = ['Country', 'Date', category])    
    
    df_new['Country'] = df_top10['Country/Region']
    df_new['Date'] = date
    df_new[category] = df_top10[date]
    df_new = df_new[::-1]
    
    ax.clear()
    
    ax.barh(df_new['Country'], df_new[category], color=["#980505","#CD1212","#D84E4E","#CB6262","#D39B5F","#F7EC10","#D0F710","#9CF710","#B4D67F","#969C8E"])
    
    dx = df_new[category].max() / 200
    
    for i, (value, name) in enumerate(zip(df_new[category], df_new['Country'])):
        ax.text(value-dx, i,     name,           size=14, weight=600, ha='right', va='bottom')
        ax.text(value+dx, i-.15, value, size=13, color='k', ha='left', va='center')
        
    ax.text(1, 0.4, date, transform=ax.transAxes, color='#777777', size=30, ha='right', weight=800)
    ax.text(0, 1.06, 'Number of Cases', transform=ax.transAxes, size=12, color='#777777')
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.1, 'The number of ' + category + ' cases up to ' + CONFIRMED_last_date,
            transform=ax.transAxes, size=24, weight=600, ha='left')
    ax.text(1, 0, 'credit @jburnmurdoch', transform=ax.transAxes, ha='right',
            color='#777777', bbox=dict(facecolor='white', alpha=0.8, edgecolor='white'))
    plt.clf()

In [ ]:
# animator = F.FuncAnimation(fig,draw_barchart,frames = numbers_confirmed.columns[3:], interval = 150, repeat = True)
# HTML(animator.to_jshtml())
# animator.save(r'confirmed_23_may.gif', writer='imagemagick', fps=60)


In [ ]:
with open('dead_23_may.gif','rb') as f: 
    display(Image(data=f.read(), format='png'))

In [ ]:
with open('recovered_23_may.gif','rb') as f: 
    display(Image(data=f.read(), format='png'))

In [ ]:
with open('confirmed_23_may.gif','rb') as f: 
    display(Image(data=f.read(), format='png'))


# Modelling

### Preparing dataset

In [ ]:
X_confirmed = pd.DataFrame(numbers_confirmed.iloc[:,4:].sum()).rename(columns = {0:'t0'})
print(X_confirmed.shape)
X_dead = pd.DataFrame(numbers_dead.iloc[:,4:].sum()).rename(columns = {0:'t0'})

In [ ]:
def shift_dataset(n, df):
    scoped_df = copy.deepcopy(df)
    
    if n <= 0:
        print('n must be > 0!')
        return
    
    for i in range(n):
        scoped_df['t' + str(i+1)] = (scoped_df['t' + str(i)].shift(-1))
        scoped_df.dropna(inplace=True)
    
    x = np.array(scoped_df.iloc[:,:n])
    y = np.array(scoped_df.iloc[:, n])
    print(scoped_df.shape)
    return x, y

In [ ]:
x_confirmed, y_confirmed = shift_dataset(4, X_confirmed)

In [ ]:
print("x_confirmed shape: {}\ny_confirmed shape: {}".format(x_confirmed.shape, y_confirmed.shape))

In [ ]:
x_dead, y_dead = shift_dataset(4, X_dead)

In [ ]:
print("x_dead shape: {}\ny_dead shape: {}".format(x_dead.shape, y_dead.shape))

# Linear Model

In [ ]:
regressor = LinearRegression()

for shift in np.arange(1, 7):
    X_confirmed_copy=copy.deepcopy(X_confirmed)
    
    x_confirmed_shift, y_confirmed_shift=shift_dataset(shift, X_confirmed_copy)
    
    print(x_confirmed_shift.shape)
    x_confirmed_log=np.log(x_confirmed_shift)
    y_confirmed_log=np.log(y_confirmed_shift)
    
    x_confirmed_log_train =  x_confirmed_log[:80-shift]
    y_confirmed_log_train =  y_confirmed_log[:80-shift]
    
    x_confirmed_log_test  = x_confirmed_log[80-shift:]
    y_confirmed_log_test  = y_confirmed_log[80-shift:]

    regressor.fit(x_confirmed_log_train, y_confirmed_log_train)
    
    y_pred=regressor.predict(x_confirmed_log_test)
    regressor.fit(x_confirmed_log_train,y_confirmed_log_train)
    y_pred=regressor.predict(x_confirmed_log_test)
    y_pred_exp=np.exp(y_pred)
    
  
    print("Prediction of confirmed cases with a shift of " + str(shift)+ " gives R2 Score of "+str(r2_score(y_pred_exp,y_confirmed_shift[80-shift:])))
    if shift==5:
        plt.plot(np.concatenate((np.exp(regressor.predict(x_confirmed_log_train)),y_pred_exp),axis=0),label='Prediction')
        plt.plot(y_confirmed_shift, label='True Values')
        plt.title("True vs Predicted Confirmed Cases")
        plt.legend(loc=1, prop={'size': 15})


In [ ]:
regressor = LinearRegression()

for shift in np.arange(1, 7):
    X_dead_copy=copy.deepcopy(X_dead)
    x_dead_shift, y_dead_shift=shift_dataset(shift, X_dead_copy)
    
    x_dead_log=np.log(x_dead_shift)
    y_dead_log=np.log(y_dead_shift)
    x_dead_log_train=x_dead_log[:80-shift]
    x_dead_log_test=x_dead_log[80-shift:]
    y_dead_log_train=y_dead_log[:80-shift]
    y_dead_log_test=y_dead_log[80-shift:]
    regressor.fit(x_dead_log_train,y_dead_log_train)
    y_pred=regressor.predict(x_dead_log_test)
    regressor.fit(x_dead_log_train,y_dead_log_train)
    y_pred=regressor.predict(x_dead_log_test)
    y_pred_exp=np.exp(y_pred)
    
  
    print("Prediction of dead cases with a shift of " + str(shift)+ " gives R2 Score of "+str(r2_score(y_pred_exp,y_dead_shift[80-shift:])))
    if shift==5:
        plt.plot(np.concatenate((np.exp(regressor.predict(x_dead_log_train)),y_pred_exp),axis=0),label='Prediction')
        plt.plot(y_dead_shift, label='True Values')
        plt.title("True vs Predicted dead Cases")
        plt.legend(loc=1, prop={'size': 15})
  

# SVM -> SVR

In [ ]:
def processDataForSVR(log_, x, y):

    if log_:
        svr_x_generic_log = np.log(x)
        svr_y_generic_log = np.log(y)
    else:
        svr_x_generic_log = x
        svr_y_generic_log = y
        
    svr_x_generic_training_log, svr_x_generic_test_log, svr_y_generic_training_log, svr_y_generic_test_log = train_test_split(svr_x_generic_log, svr_y_generic_log, test_size = 0.2, shuffle=False)
    
    return svr_x_generic_training_log, svr_y_generic_training_log, svr_x_generic_test_log, svr_y_generic_test_log
    

In [ ]:
def perform_grid_earch(model, x, y):
    grid_search = GridSearchCV(estimator=svr2,
            param_grid={
                'C': [0.1, 1],
                'epsilon': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1],
                'degree': [2, 3],            
                'coef0': [0.1, 0.01, 0.001]},
            cv=5, scoring='neg_mean_squared_error', verbose=0)
    results = grid_search.fit(x, y)
    
    print("Best results {}".format(results.best_params_))
    
#perform_grid_earch(svr2, svr_c_x_tr, svr_c_y_tr)

In [ ]:
# tr stands for trainig and te for test
# c stands for confirmed and d for dead
svr_c_x_tr, svr_c_y_tr, svr_c_x_te, svr_c_y_te = processDataForSVR(True, x_confirmed, y_confirmed)

# the most important parameter for SVR is the type of the kernel:
# - linaer
# - polynomial
# - gaussian

svr2 = SVR(kernel='poly', C=1, coef0= 0.01, degree=3, epsilon=0.05)
svr2.fit(svr_c_x_tr, svr_c_y_tr)

### svr - confirmed

In [ ]:
svr_c_y_pred = svr2.predict(svr_c_x_te)

print(svr_c_y_pred.shape)
print(svr_c_y_te.shape)

r2_score(svr_c_y_te, svr_c_y_pred)

In [ ]:
plt.plot(np.concatenate((np.exp(svr2.predict(svr_c_x_tr)), np.exp(svr_c_y_pred)),axis=0),label='Prediction')
plt.plot(y_confirmed, label='True Values')
plt.title("True vs Predicted dead Cases")
plt.legend(loc=1, prop={'size': 15})

In [ ]:
# tr stands for trainig and te for test
# c stands for confirmed and d for dead
svr_d_x_tr, svr_d_y_tr, svr_d_x_te, svr_d_y_te = processDataForSVR(True, x_dead, y_dead)

In [ ]:
svr2 = SVR(kernel='poly', C=1, coef0= 0.01, degree=3, epsilon=0.05)
svr2.fit(svr_d_x_tr, svr_d_y_tr)

svr_d_y_pred = svr2.predict(svr_d_x_te)

print(svr_c_y_pred.shape)
print(svr_c_y_te.shape)

r2_score(svr_d_y_te, svr_d_y_pred)

In [ ]:
plt.rcParams['figure.figsize'] = [10,10]
plt.plot(np.concatenate((np.exp(svr2.predict(svr_d_x_tr)), np.exp(svr_d_y_pred)),axis=0),label='Prediction')
plt.plot(y_dead, label='True Values')
plt.title("True vs Predicted dead Cases")
plt.legend(loc=1, prop={'size': 15})

In [ ]:
n = 1

x_confirmed, y_confirmed = shift_dataset(n, X_confirmed)

In [ ]:
x_confirmed_train, x_confirmed_test, y_confirmed_train, y_confirmed_test = train_test_split(x_confirmed,
                                                                                            y_confirmed, test_size = 0.2)
x_confirmed_train = x_confirmed_train.reshape(x_confirmed_train.shape[0],n)
x_confirmed_test = x_confirmed_test.reshape(x_confirmed_test.shape[0],n)

In [ ]:
x_dead, y_dead = shift_dataset(n, X_dead)

In [ ]:
covid_worldwide = pd.DataFrame({'Confirmed' : [], 'Dead' : [], 'Recovered' : []})
covid_worldwide['Confirmed'] = numbers_confirmed.iloc[:,4:].sum()
covid_worldwide['Dead'] = numbers_dead.iloc[:,4:].sum()
covid_worldwide['Recovered'] = numbers_recovered.iloc[:,4:].sum()

In [ ]:
def forecast(x, model, n, scaler):
    
    
    # n = number of days ahead to forecast
    # x =last confirmed/dead/recovered value
    # model = lasso/lstm etc.
    
    y_pred = []
    curr_value = scaler.transform(x)
    
    for i in range(n):
        val = model.predict(curr_value).reshape(-1,1)
        y_pred.append(val[0,0])
        curr_value = scaler.transform(val)
        
    
    return y_pred
    

In [ ]:
def forecast_poly(x, model, n, poly_features, scaler):
    
    
    # n = number of days ahead to forecast
    # x =last confirmed/dead/recovered value
    # model = lasso/lstm etc.
    
    y_pred = []
    scaled = scaler.transform(x)
    curr_value = poly_features.fit_transform(scaled).reshape(1,-1)
    
    for i in range(n):
        val = model.predict(curr_value)[0]
        y_pred.append(val)
        
        
        
        curr_value = poly_features.fit_transform(scaler.transform(val.reshape(1,1))).reshape(1,-1)
        
    
    return y_pred

In [ ]:
def forecast_lstm(x, model, n, scaler):
    
    
    # n = number of days ahead to forecast
    # x =last confirmed/dead/recovered value
    # model = lasso/lstm etc.
    
    y_pred = []
    curr_value = scaler.transform(x).reshape(1,1,-1)
    
    for i in range(n):
        val = model.predict(curr_value)
        y_pred.append(val[0,0])
        curr_value = scaler.transform(val).reshape(1,1,-1)
        
    
    return y_pred

### Forecast variables

In [ ]:
last_conf_value = covid_worldwide['Confirmed'].iloc[-1].reshape(1,1)
days = np.arange(covid_worldwide.shape[0])

## Normalization

In [ ]:
scaler = StandardScaler()
x_confirmed_train = scaler.fit_transform(x_confirmed_train)
x_confirmed_test = scaler.transform(x_confirmed_test)

## Polynomial Regression

In [ ]:
poly_features = PolynomialFeatures(degree = 5)

In [ ]:
x_confirmed_poly = poly_features.fit_transform(x_confirmed_train)
x_conf_test_poly = poly_features.fit_transform(x_confirmed_test)

In [ ]:
reg = LinearRegression().fit(x_confirmed_poly, y_confirmed_train)

In [ ]:
r2_training = reg.score(x_confirmed_poly, y_confirmed_train)
mse_training = mean_squared_error(y_confirmed_train, reg.predict(x_confirmed_poly))
r2_test = reg.score(x_conf_test_poly, y_confirmed_test)
mse_testing = mean_squared_error(y_confirmed_test, reg.predict(x_conf_test_poly))
print(r2_training,mse_training, r2_test, mse_testing)

<table>
  <tr>
    <th></th>
    <th scope="col">MSE</th>
    <th scope="col">$R^2$</th>
  </tr>
  <tr>
    <th scope="row">Training</th>
    <td>39136302.61040542</td>
    <td>0.999986</td>
  </tr>
  <tr>
    <th scope="row">Testing</th>
    <td>47674661.12815987</td>
    <td>0.999989 </td>
  </tr>

</table>

### Forecast - polynomial regression

In [ ]:
y_forecast_preg = forecast_poly(last_conf_value, reg, 10, poly_features, scaler)
y_forecast_preg.insert(0, last_conf_value[0,0])

days_forecast_poly = np.arange(len(y_forecast_preg)) + days[-1]


In [ ]:
fig, ax = plt.subplots()


ax.plot(days,covid_worldwide['Confirmed'], label = "Actual")
ax.plot(days_forecast_poly, y_forecast_preg, label = "Forecast")

ax.set_title('Polynomial regression - forecasting the next {} days'.format(len(days_forecast_poly)-1))
ax.set_xlabel("Day")
ax.set_ylabel("# of cases")
ax.legend(loc = 'best')

plt.rcParams['figure.figsize'] = [18, 12]
plt.show()

## LASSO

In [ ]:
lasso_reg = Lasso()
lasso_params = {'alpha':[0.01, 0.1, 1, 10], 
                'tol' : [1e-4, 1e-3, 1e-2], 
                'selection' : ['cyclic', 'random']
               }

cv = [(slice(None), slice(None))]

gs_lasso = GridSearchCV(estimator=lasso_reg, param_grid=lasso_params, 
                  scoring=make_scorer(mean_squared_error, greater_is_better = False), cv=cv, n_jobs=-1)

gs_lasso.fit(x_confirmed_train, y_confirmed_train)

In [ ]:
gs_lasso.best_params_

In [ ]:
lasso_reg_best = Lasso(alpha = 0.01, selection = 'cyclic', tol = 0.0001)
lasso_reg_best.fit(x_confirmed_train, y_confirmed_train)

In [ ]:
r2_training_lasso = lasso_reg_best.score(x_confirmed_train, y_confirmed_train)
mse_training_lasso = mean_squared_error(y_confirmed_train, lasso_reg_best.predict(x_confirmed_train))
r2_test_lasso = lasso_reg_best.score(x_confirmed_test, y_confirmed_test)
mse_testing_lasso = mean_squared_error(y_confirmed_test, lasso_reg_best.predict(x_confirmed_test))
print(r2_training_lasso,mse_training_lasso, r2_test_lasso, mse_testing_lasso)

<table>
  <tr>
    <th></th>
    <th scope="col">MSE</th>
    <th scope="col">$R^2$</th>
  </tr>
  <tr>
    <th scope="row">Training</th>
    <td>422634914.04812187</td>
    <td>0.999838</td>
  </tr>
  <tr>
    <th scope="row">Testing</th>
    <td>351981514.9093248</td>
    <td>0.999847</td>
  </tr>

</table>

### Forecast - LASSO

In [ ]:
y_forecast_lasso = forecast(last_conf_value, lasso_reg_best, 10, scaler)
y_forecast_lasso.insert(0, last_conf_value[0,0])

In [ ]:
fig, ax = plt.subplots()

ax.plot(days,covid_worldwide['Confirmed'], label = "Actual")
ax.plot(days_forecast_poly, y_forecast_lasso, label = "Forecast")

ax.set_title('LASSO - forecasting the next {} days'.format(len(days_forecast_poly)-1))
ax.set_xlabel("Day")
ax.set_ylabel("# of cases")
ax.legend(loc = 'best')

plt.rcParams['figure.figsize'] = [18, 12]
plt.show()

## LSTM

In [ ]:
x_confirmed_train_lstm = x_confirmed_train.reshape(x_confirmed_train.shape[0], 1, n)

In [ ]:
model_lstm = Sequential([
    LSTM(20, activation = 'relu', input_shape = (1,n), return_sequences = True),
    LSTM(20, activation = 'relu'),
    Dense(1)
])

model_lstm.compile(optimizer = Adam(lr = 1.1), loss = 'mse')

In [ ]:
model_lstm.summary()

In [ ]:
history = model_lstm.fit(x_confirmed_train_lstm, y_confirmed_train, epochs = 100)

In [ ]:
x_confirmed_test_lstm = x_confirmed_test.reshape(x_confirmed_test.shape[0], 1, n)

In [ ]:
prediction_lstm = model_lstm.predict(x_confirmed_test_lstm)

In [ ]:
r2_training_lstm = r2_score(y_confirmed_train, model_lstm.predict(x_confirmed_train_lstm))
r2_test_lstm = r2_score(y_confirmed_test, prediction_lstm)
mse_testing_lstm = mean_squared_error(y_confirmed_test, prediction_lstm)
print(r2_training_lstm, r2_test_lstm, mse_testing_lstm)

<table>
  <tr>
    <th></th>
    <th scope="col">MSE</th>
    <th scope="col">$R^2$</th>
  </tr>
  <tr>
    <th scope="row">Training</th>
    <td>407080714.6667</td>
    <td>0.999837</td>
  </tr>
  <tr>
    <th scope="row">Testing</th>
    <td>359227875.00750625</td>
    <td>0.999845</td>
  </tr>

</table>

In [ ]:
y_forecast_lstm = forecast_lstm(last_conf_value, model_lstm, 10, scaler)
y_forecast_lstm.insert(0, last_conf_value[0,0])

In [ ]:
y_forecast_lstm

In [ ]:
fig, ax = plt.subplots()

ax.plot(days,covid_worldwide['Confirmed'], label = "Actual")
ax.plot(days_forecast_poly, y_forecast_lstm, label = "Forecast")

ax.set_title('LSTM - forecasting the next {} days'.format(len(days_forecast_poly)-1))
ax.set_xlabel("Day")
ax.set_ylabel("# of cases")
ax.legend(loc = 'best')

plt.rcParams['figure.figsize'] = [18, 12]
plt.show()